In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import seaborn as sns
from sklearn import linear_model

# Load Fama French Factors

Downloaded Fama-French model from [French's website](http://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html#Developed) around June 16, 2019:

- [Details](http://mba.tuck.dartmouth.edu/pages/faculty/ken.french/Data_Library/f-f_factors.html)
- [File](http://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/F-F_Research_Data_Factors_CSV.zip)

The file gives monthly returns initially and annual returns at the end.

In [2]:
df_factors = pd.read_csv("data/FF-Factors.csv", skiprows=1120, skipfooter=1, engine='python').rename({'Unnamed: 0': "Year"}, axis=1)
df_factors[['Mkt-RF','SMB','HML','RF']] = df_factors[['Mkt-RF','SMB','HML','RF']] / 100.
df_factors.head()

,Year,Mkt-RF,SMB,HML,RF
0,1927,0.2947,-0.0246,-0.0375,0.0312
1,1928,0.3539,0.0420,-0.0615,0.0356
2,1929,-0.1954,-0.3080,0.1181,0.0475
3,1930,-0.3123,-0.0513,-0.1228,0.0241
4,1931,-0.4511,0.0353,-0.1429,0.0107


# Load Russell Smart Weight Index

In [3]:
df_russell = pd.read_csv("data/Russell.tsv", sep='\t').rename({'Unnamed: 0': "Year"}, axis=1)

for col in ('Smart Weighting', 'S&P 500'):
    df_russell[col] = (
        df_russell[col]
            .str.rstrip('%')
            .astype(float)
            .pipe(lambda x: x/100.)
    )
    
df_russell.head()

,Year,Smart Weighting,S&P 500
0,1991,0.335,0.306
1,1992,0.118,0.077
2,1993,0.133,0.100
3,1994,0.022,0.013
4,1995,0.368,0.374


In [4]:
df = df_russell.merge(df_factors, on='Year', how='inner')
df.head()

,Year,Smart Weighting,S&P 500,Mkt-RF,SMB,HML,RF
0,1991,0.335,0.306,0.2918,0.1613,-0.1441,0.0560
1,1992,0.118,0.077,0.0623,0.0758,0.2428,0.0351
2,1993,0.133,0.100,0.0821,0.0580,0.1890,0.0290
3,1994,0.022,0.013,-0.0410,-0.0107,-0.0066,0.0390
4,1995,0.368,0.374,0.3122,-0.0910,0.0524,0.0560


# Excess Return of Smart Weighting

In [5]:
(df['Smart Weighting'] - df['S&P 500']).mean()

0.013666666666666674

# Fama French Factors

In [6]:
factors=['Mkt-RF','SMB','HML']

def ff_decomposition(df, returns):
    model = linear_model.LinearRegression()
    model.fit(df[factors], returns - df['RF'])
    return pd.Series(list(model.coef_) + [model.intercept_], index=factors + ['Alpha'])

ff_weights = pd.DataFrame({
    col: ff_decomposition(df, df[col])
    for col in ('Smart Weighting', 'S&P 500')
})
ff_weights.head()

,Smart Weighting,S&P 500
Mkt-RF,0.954591,0.990379
SMB,0.052521,-0.159506
HML,0.156031,0.041177
Alpha,0.005723,-0.000011
